# Heather's Capstone Notebook

In [3]:
import pandas as pd
import numpy as np

import new_wrangle as w
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
# acquire dataset

df = w.wrangle_walmart()

In [5]:
df.head()

,store_id,weekly_sales,holiday_flag,temperature,fuel_price,CPI,unemployment,type,store_size,month,year,quarter,deflated_series,sales_delta,season
Date,,,,,,,,,,,,,,,
2010-01-10,5,283178.12,0,71,2.60,212.227,6.768,B,34875,January,2010,1,1334.32,-9953.46,Winter
2010-01-10,15,566945.95,0,59,2.84,132.757,8.067,B,123737,January,2010,1,4270.55,18403.48,Winter
2010-01-10,42,481523.93,0,86,3.00,126.235,9.003,C,39690,January,2010,1,3814.50,-10739.03,Winter
2010-01-10,33,224294.39,0,91,3.00,126.235,9.265,A,39690,January,2010,1,1776.80,-7682.45,Winter
2010-01-10,36,422169.47,0,74,2.57,210.440,8.476,A,39910,January,2010,1,2006.13,2820.88,Winter


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6435 entries, 2010-01-10 to 2012-12-10
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   store_id         6435 non-null   int64  
 1   weekly_sales     6435 non-null   float64
 2   holiday_flag     6435 non-null   int64  
 3   temperature      6435 non-null   int64  
 4   fuel_price       6435 non-null   float64
 5   CPI              6435 non-null   float64
 6   unemployment     6435 non-null   float64
 7   type             6435 non-null   object 
 8   store_size       6435 non-null   int64  
 9   month            6435 non-null   object 
 10  year             6435 non-null   int64  
 11  quarter          6435 non-null   int64  
 12  deflated_series  6435 non-null   float64
 13  sales_delta      6390 non-null   float64
 14  season           6435 non-null   object 
dtypes: float64(6), int64(6), object(3)
memory usage: 804.4+ KB


## Add Holiday's to column

Goal: Create variable called `holiday_week` with the name of the holiday. Later will break out into `is_superbowl`, `is_laborday`, `is_christmas` etc. no holiday

### Holiday List

Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13

Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13

Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13

Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

In [7]:
# get value counts of weeks with holidays and weeks without
df.holiday_flag.value_counts()

0    5985
1     450
Name: holiday_flag, dtype: int64

There's already a column called holiday flag. 0 no holiday 1 holiday. Since we already have this information, we need to know which of these weeks belong to what projects. 

In [26]:
# OOOHHH Those weeks are already there. So Need to filter for those 

christmases = ['2010-12-31', '2011-12-30', '2012-12-28']

super_bowls = ['2010-02-12', '2011-02-11', '2012-02-10']

labor_days = ['2010-09-10', '2011-09-09', '2012-09-07']

thanksgivings = ['2010-11-26', '2011-11-25', '2012-11-23']


In [ ]:
# Create loopy deal

def add_which_holiday(df, dates_list):
    '''
    This function takes in a dataframe
    And a string list of dates
    Turnes that list of dates into a date time 
    
    '''
    dates_list = [dt.datetime.strptime(date, "%Y-%m-%d").date() for date in christmases]


In [27]:
# christmas
dates_list = [dt.datetime.strptime(date, "%Y-%m-%d").date() for date in christmases]
    
df.loc[df.index.isin(dates_list) == True, 'holiday_name'] = 'christmas'

#super bowl
dates_list = [dt.datetime.strptime(date, "%Y-%m-%d").date() for date in super_bowls]  
df.loc[df.index.isin(dates_list) == True,'holiday_name'] = 'super_bowl'

# labor day
dates_list = [dt.datetime.strptime(date, "%Y-%m-%d").date() for date in labor_days]  
df.loc[df.index.isin(dates_list) == True, 'holiday_name'] = 'labor_day'

# thanksgiving
dates_list = [dt.datetime.strptime(date, "%Y-%m-%d").date() for date in thanksgivings]  
df.loc[df.index.isin(dates_list) == True, 'holiday_name'] = 'thanksgiving'

In [28]:
df.holiday_name.value_counts()

thanksgiving    90
christmas       90
labor_day       45
Name: holiday_name, dtype: int64

In [29]:
sb_list = [dt.datetime.strptime(date, "%Y-%m-%d").date() for date in super_bowls]

df.loc[df.index.isin(sb_list)]

,store_id,weekly_sales,holiday_flag,temperature,fuel_price,CPI,unemployment,type,store_size,month,year,quarter,deflated_series,sales_delta,season,holiday_name
Date,,,,,,,,,,,,,,,,


In [ ]:
plt.figure(figsize = (15,7))
df[df.store == 1]['weekly_sales'].plot()
plt.axhline(df[df.store == 1]['weekly_sales'].mean())
plt.show()

In [ ]:
super_bowls = ['2010-02-12', '2011-02-11', '2012-02-10']

In [46]:
df.groupby(df.index).store_id.count().head(55)

Date
2010-01-10    45
2010-02-04    45
2010-02-07    45
2010-02-19    45
2010-02-26    45
2010-03-09    45
2010-03-12    45
2010-03-19    45
2010-03-26    45
2010-04-06    45
2010-04-16    45
2010-04-23    45
2010-04-30    45
2010-05-02    45
2010-05-03    45
2010-05-11    45
2010-05-14    45
2010-05-21    45
2010-05-28    45
2010-06-08    45
2010-06-18    45
2010-06-25    45
2010-07-05    45
2010-07-16    45
2010-07-23    45
2010-07-30    45
2010-08-10    45
2010-08-13    45
2010-08-20    45
2010-08-27    45
2010-09-04    45
2010-09-07    45
2010-09-17    45
2010-09-24    45
2010-10-09    45
2010-10-12    45
2010-10-15    45
2010-10-22    45
2010-10-29    45
2010-11-06    45
2010-11-19    45
2010-11-26    45
2010-12-02    45
2010-12-03    45
2010-12-11    45
2010-12-17    45
2010-12-24    45
2010-12-31    45
2011-01-04    45
2011-01-07    45
2011-01-14    45
2011-01-21    45
2011-01-28    45
2011-02-09    45
2011-02-12    45
Name: store_id, dtype: int64

In [48]:
walmart = pd.read_csv('walmart_sales.csv')

In [51]:
walmart.Date.value_counts().sort_index().head(20)

01-04-2011    45
01-06-2012    45
01-07-2011    45
01-10-2010    45
02-03-2012    45
02-04-2010    45
02-07-2010    45
02-09-2011    45
02-12-2011    45
03-02-2012    45
03-06-2011    45
03-08-2012    45
03-09-2010    45
03-12-2010    45
04-02-2011    45
04-03-2011    45
04-05-2012    45
04-06-2010    45
04-11-2011    45
05-02-2010    45
Name: Date, dtype: int64

In [52]:
pd.to_datetime(walmart.Date, dayfirst=True)


0      2010-02-05
1      2010-02-12
2      2010-02-19
3      2010-02-26
4      2010-03-05
          ...    
6430   2012-09-28
6431   2012-10-05
6432   2012-10-12
6433   2012-10-19
6434   2012-10-26
Name: Date, Length: 6435, dtype: datetime64[ns]